In [2]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
%matplotlib qt 
import seaborn as sns
import nltk
import plotly.express as px
import bar_chart_race as bcr
from wordcloud import WordCloud
from textblob import TextBlob
from sklearn.metrics import confusion_matrix

nltk.download('stopwords')
nltk.download('wordnet')



from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from matplotlib.animation import FuncAnimation
plt.style.use('ggplot')
import matplotlib
from sklearn.model_selection import train_test_split, GridSearchCV



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/niveditasaha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/niveditasaha/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Details of the dataset

In [3]:

# Reading the csv file
twitter_post = pd.read_csv('/Users/niveditasaha/Downloads/COURSEWORK 2 FOR viz/text.csv', encoding='ISO-8859-1')

# Display the first few rows of the dataframe
print(twitter_post.head())


   ID                                               text  label
0   0      i just feel really helpless and heavy hearted      4
1   1  ive enjoyed being able to slouch about relax a...      0
2   2  i gave up my internship with the dmrg and am f...      4
3   3                         i dont know i feel so lost      0
4   4  i am a kindergarten teacher and i am thoroughl...      4


In [4]:
twitter_post.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 416809 entries, 0 to 416808
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   ID      416809 non-null  int64 
 1   text    416809 non-null  object
 2   label   416809 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 9.5+ MB


In [5]:
# Number of rows and columns
num_rows, num_columns = twitter_post.shape
# Print the number of rows and columns
print("Number of rows:", num_rows)
print("Number of columns:", num_columns)


Number of rows: 416809
Number of columns: 3


In [6]:
twitter_post.columns

Index(['ID', 'text', 'label'], dtype='object')

In [7]:
message_df = twitter_post.drop(['ID', 'label'], axis=1)
message_df.head()


text
0      i just feel really helpless and heavy hearted
1  ive enjoyed being able to slouch about relax a...
2  i gave up my internship with the dmrg and am f...
3                         i dont know i feel so lost
4  i am a kindergarten teacher and i am thoroughl...

In [8]:

# Checking the number of rows in the DataFrame
print("Number of rows in message_df:", len(message_df))

# Accessing the first 5 rows of the 'text' column
for i in range(min(5, len(message_df))):
    print(message_df['text'].iloc[i], "\n")


Number of rows in message_df: 416809
i just feel really helpless and heavy hearted 

ive enjoyed being able to slouch about relax and unwind and frankly needed it after those last few weeks around the end of uni and the expo i have lately started to find myself feeling a bit listless which is never really a good thing 

i gave up my internship with the dmrg and am feeling distraught 

i dont know i feel so lost 

i am a kindergarten teacher and i am thoroughly weary of my job after having taken the university entrance exam i suffered from anxiety for weeks as i did not want to carry on with my work studies were the only alternative 



In [9]:
# Check for missing values
missing_values = twitter_post.isnull().sum()
# Print the number of missing values for each column
print("Missing values in each column:")
print(missing_values)


Missing values in each column:
ID       0
text     0
label    0
dtype: int64


In [10]:
# Check the distribution of the label column
label_distribution = twitter_post['label'].value_counts()

# Print the distribution of the label column
print("Distribution of the label column:")
print(label_distribution)


Distribution of the label column:
label
1    141067
0    121187
3     57317
4     47712
2     34554
5     14972
Name: count, dtype: int64


# Classify the data

In [11]:
def polarity(text):
    return TextBlob(text).sentiment.polarity


In [12]:
message_df['polarity'] = message_df['text'].apply(polarity)
message_df.head(10)

text  polarity
0      i just feel really helpless and heavy hearted  0.000000
1  ive enjoyed being able to slouch about relax a...  0.025000
2  i gave up my internship with the dmrg and am f... -0.600000
3                         i dont know i feel so lost  0.000000
4  i am a kindergarten teacher and i am thoroughl...  0.000000
5         i was beginning to feel quite disheartened  0.000000
6  i would think that whomever would be lucky eno...  0.208333
7  i fear that they won t ever feel that deliciou...  0.233333
8  im forever taking some time out to have a lie ... -0.327778
9  i can still lose the weight without feeling de...  0.000000

In [13]:
def sentiment(label):
    if label <0:
        return "Negative"
    elif label ==0:
        return "Neutral"
    elif label>0:
        return "Positive"

In [14]:
message_df['sentiment'] = message_df['polarity'].apply(sentiment)
message_df.head()

text  polarity sentiment
0      i just feel really helpless and heavy hearted     0.000   Neutral
1  ive enjoyed being able to slouch about relax a...     0.025  Positive
2  i gave up my internship with the dmrg and am f...    -0.600  Negative
3                         i dont know i feel so lost     0.000   Neutral
4  i am a kindergarten teacher and i am thoroughl...     0.000   Neutral

In [15]:

# Set the colors for the pie chart
colors = sns.color_palette("viridis", len(message_df['sentiment'].unique()))

# Set the explode values to emphasize a particular sentiment
explode = (0.1, 0, 0)  # explode the first slice (positive sentiment)

# Create a pie chart with shadow and explode effect
plt.figure(figsize=(10, 8))
plt.pie(message_df['sentiment'].value_counts(), labels=message_df['sentiment'].value_counts().index, autopct='%1.1f%%', colors=colors, startangle=140, explode=explode, shadow=True, wedgeprops=dict(width=0.4))
plt.title('Sentiment Distribution (Pie Chart)', fontsize=16)
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle

# Add a legend
plt.legend(title="Sentiment", loc="upper right", fontsize='medium')

plt.show()


In [16]:
category_dict = {0: "Sadness", 1: "Joy", 2: "Love", 3: "Anger", 4: "Fear", 5: "Surprise"}

twitter_post["Category"] = twitter_post["label"].map(category_dict)


In [17]:

# Get label counts
label_counts = twitter_post["Category"].value_counts()

# Plot the bar chart
plt.figure(figsize=(10, 6))
sns.barplot(x=label_counts.index, y=label_counts.values, palette='viridis')
plt.title('Label Distribution')
plt.xlabel('Category')
plt.ylabel('Frequency')
plt.grid(True)
plt.xticks(rotation=45)
plt.show()


In [18]:
# labels and counts variables 
labels = twitter_post["Category"].value_counts().index
counts = twitter_post["Category"].value_counts().values


# Create figure and axes
fig, ax = plt.subplots(figsize=(8, 6))

# Function to update the plot
def update(frame):
    ax.clear()
    ax.bar(labels[:frame], counts[:frame], color='black')
    ax.set_title('Label Distribution')
    ax.set_xlabel('Category')
    ax.set_ylabel('Frequency')
    ax.set_xticklabels(labels[:frame], rotation=1, ha='center')
    ax.grid(True)

# Animate the plot
ani = FuncAnimation(fig, update, frames=len(labels), repeat=False)

# Display the animation
plt.show()


/var/folders/kr/957l0hy92fl69nkxf94vkfdc0000gn/T/ipykernel_1912/1302225278.py:16: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(labels[:frame], rotation=1, ha='center')


# Data pre-processing

In [19]:
import random

duplicate_list = twitter_post[twitter_post.duplicated("text")]["text"].to_list()
print(f"Total duplicates in the data: {len(duplicate_list)}")


Total duplicates in the data: 22987


In [20]:
duplicate_list_s = random.sample(duplicate_list,1)
twitter_post[twitter_post["text"]==duplicate_list_s[0]]

ID                              text  label Category
149744  149744  i feel hated helping prevent gay      0  Sadness
322284  322284  i feel hated helping prevent gay      3    Anger

In [21]:
# Removing duplicates
twitter_post.drop_duplicates("text",inplace=True)
print(f"Shape of the data after removing duplicates: {twitter_post.shape}")

Shape of the data after removing duplicates: (393822, 4)


In [22]:
# Tokenize 


nltk.download('punkt')


twitter_post["c_text"] = twitter_post["text"].apply(lambda x: word_tokenize(x))


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/niveditasaha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [23]:
# Function to perform negation detection and tokenization
def tokenize_with_negation_detection(text):
    # Tokenize the text
    words = word_tokenize(text)
    negation = False
    # List to store modified tokens with negation suffix
    modified_words = []
    for word in words:
        # Check for negation words and set the flag
        if word.lower() in ['not', 'no', 'never']:
            negation = True
        # If negation flag is set, append '_NEG' suffix to the word
        if negation:
            modified_words.append(word + '_NEG')
        else:
            modified_words.append(word)
        # Reset negation flag after encountering punctuation or end of sentence
        if re.match(r'[.!?]', word):
            negation = False
    return modified_words


In [24]:
# Apply negation detection and tokenization to the 'text' column
twitter_post["c_text_with_negation"] = twitter_post["text"].apply(tokenize_with_negation_detection)

# Display the first few rows of the dataframe with modified tokens
print(twitter_post[["text", "c_text_with_negation"]].head())


                                                text  \
0      i just feel really helpless and heavy hearted   
1  ive enjoyed being able to slouch about relax a...   
2  i gave up my internship with the dmrg and am f...   
3                         i dont know i feel so lost   
4  i am a kindergarten teacher and i am thoroughl...   

                                c_text_with_negation  
0  [i, just, feel, really, helpless, and, heavy, ...  
1  [ive, enjoyed, being, able, to, slouch, about,...  
2  [i, gave, up, my, internship, with, the, dmrg,...  
3                 [i, dont, know, i, feel, so, lost]  
4  [i, am, a, kindergarten, teacher, and, i, am, ...  


In [25]:

# Example of a single text with negation detection
print("Original text:", twitter_post["text"].iloc[0])
print("Tokens with negation detection:", twitter_post["c_text_with_negation"].iloc[0])



Original text: i just feel really helpless and heavy hearted
Tokens with negation detection: ['i', 'just', 'feel', 'really', 'helpless', 'and', 'heavy', 'hearted']


In [26]:
# Function to count the number of negations in a list of tokens
def count_negations(tokens):
    # Count the number of tokens containing '_NEG' suffix
    negation_count = sum(1 for token in tokens if '_NEG' in token)
    return negation_count

# Apply the function to the 'c_text_with_negation' column
twitter_post["negation_count"] = twitter_post["c_text_with_negation"].apply(count_negations)

# Display the first few rows of the dataframe with negation counts
print(twitter_post[["text", "negation_count"]].head())


                                                text  negation_count
0      i just feel really helpless and heavy hearted               0
1  ive enjoyed being able to slouch about relax a...               5
2  i gave up my internship with the dmrg and am f...               0
3                         i dont know i feel so lost               0
4  i am a kindergarten teacher and i am thoroughl...              13


In [27]:


# Function to get antonyms of a word using WordNet
def get_antonyms(word):
    antonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            for antonym in lemma.antonyms():
                antonyms.add(antonym.name())
    return list(antonyms)

# Function to handle negations in a sentence
def handle_negations(sentence):
    # Tokenize the sentence
    words = nltk.word_tokenize(sentence)
    # Initialize a list to store the modified words
    modified_words = []
    # Initialize a flag to check if the current word is negated
    negation_flag = False
    # Iterate through the words in the sentence
    for word in words:
        # Check if the word is a negation word
        if word.lower() in ['not', "n't", "no", "never"]:
            negation_flag = not negation_flag
        else:
            # If the word is negated, find its antonym and append it to the modified words list
            if negation_flag:
                antonyms = get_antonyms(word)
                if antonyms:
                    modified_words.append(antonyms[0])  # Assume the first antonym as the replacement
                else:
                    modified_words.append("not_" + word)  # If no antonym found, prepend "not_" to the word
            else:
                modified_words.append(word)
    # Join the modified words to form the new sentence
    modified_sentence = ' '.join(modified_words)
    return modified_sentence

# Example sentence with negations
sentence = "I am not happy with the service. It's not good at all."

# Handle negations in the sentence
modified_sentence = handle_negations(sentence)

# Print the original and modified sentences
print("Original sentence:", sentence)
print("Modified sentence:", modified_sentence)


Original sentence: I am not happy with the service. It's not good at all.
Modified sentence: I am unhappy not_with not_the disservice not_. not_It not_'s good at all .


In [28]:
# Stopwords

print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [29]:
# Get English stopwords from NLTK
stop_words = set(stopwords.words('english'))

# Define custom stopwords if needed
custom_stop_words = {'custom', 'stop', 'words', 'here'}

# Concatenate NLTK stopwords with custom stopwords
com_stop_words = stop_words.union(custom_stop_words)

print(len(com_stop_words))
print(len(set(com_stop_words)))


182
182


In [30]:

from nltk.stem.porter import PorterStemmer

# Initialize Porter Stemmer
port_stem = PorterStemmer()

# Function for stemming and removing stopwords
def stemming_and_remove_stopwords(content):
    # Remove everything other than words
    content = re.sub('[^a-zA-Z]',' ', content)
    # Split the content into words using whitespace as delimiter
    words = content.lower().split()
    # Stemming and remove stopwords
    stemmed_words = [port_stem.stem(word) for word in words if word not in com_stop_words]

    # Join the stemmed words back into a string
    stemmed_content = ' '.join(stemmed_words)
    return stemmed_content

# Apply stemming and remove stopwords to the 'text' column and create a new column 'stemmed_content'
twitter_post['stemmed_content'] = twitter_post['text'].apply(stemming_and_remove_stopwords)



In [31]:
# Display the first few rows of the dataframe
print(twitter_post.head())


   ID                                               text  label Category  \
0   0      i just feel really helpless and heavy hearted      4     Fear   
1   1  ive enjoyed being able to slouch about relax a...      0  Sadness   
2   2  i gave up my internship with the dmrg and am f...      4     Fear   
3   3                         i dont know i feel so lost      0  Sadness   
4   4  i am a kindergarten teacher and i am thoroughl...      4     Fear   

                                              c_text  \
0  [i, just, feel, really, helpless, and, heavy, ...   
1  [ive, enjoyed, being, able, to, slouch, about,...   
2  [i, gave, up, my, internship, with, the, dmrg,...   
3                 [i, dont, know, i, feel, so, lost]   
4  [i, am, a, kindergarten, teacher, and, i, am, ...   

                                c_text_with_negation  negation_count  \
0  [i, just, feel, really, helpless, and, heavy, ...               0   
1  [ive, enjoyed, being, able, to, slouch, about,...          

In [32]:
# Print the original text
print("Original text:")
print(twitter_post['text'].iloc[0])

Original text:
i just feel really helpless and heavy hearted


In [33]:
# Print the stemmed and filtered text
print("\nStemmed and filtered text:")
print(twitter_post['stemmed_content'].iloc[0])



Stemmed and filtered text:
feel realli helpless heavi heart


In [34]:
# Define the list of custom stopwords and negation words
custom_stopwords = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
negation_words = ['no', 'not', 'never', 'none', 'nobody', 'nowhere', 'nothing', 'neither', 'nor', 'n\'t']

# Tokenize the text into individual words
words = twitter_post['stemmed_content'].apply(str.split)

# Initialize an empty list to store non-stopwords
filtered_text_list = []

# Iterate through each list of words
for word_list in words:
    # Initialize an empty list to store filtered words for this text
    filtered_word_list = []
    # Iterate through each word in the list
    for word in word_list:
        # Check if the word is a stopword and not a negation word
        if word.lower() in stopwords.words('english') and word.lower() not in negation_words:
            # Skip this word
            continue
        # If the word is not a stopword or it's a negation word, keep it
        filtered_word_list.append(word)
    # Join the filtered word list to form a string and append it to the overall list of filtered words
    filtered_text_list.append(' '.join(filtered_word_list))

# Join the list of filtered texts into a single string
filtered_text = ' '.join(filtered_text_list)





In [35]:
# Print the filtered text
print("\nFiltered text:")
print('\n'.join(filtered_text.split('\n')[:10]))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# Data Visualization

In [36]:
# DATA VISUALIZATION

twitter_post["Category"].value_counts().to_frame().style.background_gradient("Greens")

In [37]:
labels = twitter_post["Category"].value_counts().index
counts = twitter_post["Category"].value_counts().values 

In [38]:
category_counts = twitter_post["Category"].value_counts()
# Plot the pie chart
plt.figure(figsize=(4, 4))
plt.pie(category_counts, labels=category_counts.index, autopct='%1.1f%%', colors=sns.color_palette("viridis", len(category_counts)))
plt.title('Label Distribution (Pie Chart)')
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle
plt.show()


In [39]:
# Create a single figure with subplots for each category
plt.figure(figsize=(15, 10))

# Get unique categories from the "Category" column
categories = twitter_post["Category"].unique()

# Define the number of rows and columns for subplots
num_rows = 2
num_cols = 3

# Iterate over each category and subplot index
for i, category in enumerate(categories, start=1):
    # Filter the dataframe for the current category
    category_df = twitter_post[twitter_post["Category"] == category]
    
    # Combine text from all rows in the category into a single string
    category_text = ' '.join(category_df['stemmed_content'])
    
    # Generate WordCloud for the current category
    wordcloud = WordCloud(width=400, height=400, background_color='black', stopwords=com_stop_words, min_font_size=10).generate(category_text)
    
    # Add subplot for the current category
    plt.subplot(num_rows, num_cols, i)
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(f'{category}')
    plt.axis("off")

# Adjust layout to prevent overlap
plt.tight_layout()

# Show the plot
plt.show()


# Develop a model by using ML

In [40]:


X = twitter_post['stemmed_content']  # Input features (preprocessed text)
y = twitter_post['label']            # Target variable (category labels)

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorization (using TF-IDF)
tfidf_vectorizer = TfidfVectorizer(max_features=4000)  # adjust the max_features parameter as needed
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)






In [41]:
# Model Training


model = LogisticRegression(max_iter=1000)  # Increase max_iter to avoid convergence warning
model.fit(X_train_tfidf, y_train)


LogisticRegression(max_iter=1000)

In [42]:
# Model Evaluation

y_pred = model.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)


Accuracy: 0.9066971370532597
Confusion Matrix:
 [[22152   508   133   484   295    68]
 [  602 25290   601   240   225   148]
 [  215  1059  4420    67    38    16]
 [  445   295    55  9963   181    20]
 [  386   270    56   209  7683   178]
 [   82   194    17    31   231  1908]]


In [43]:

# Plot the confusion matrix as a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()


In [ ]:
# Hyperparameter Tuning
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.1, 1, 10], 'penalty': ['l2']}
grid_search = GridSearchCV(LogisticRegression(max_iter=1000), param_grid, cv=5)
grid_search.fit(X_train_tfidf, y_train)

best_model = grid_search.best_estimator_
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score:", grid_search.best_score_)


In [54]:

# Define the parameter grid and perform GridSearchCV
param_grid = {'C': [0.1, 1, 10], 'penalty': ['l2']}
grid_search = GridSearchCV(LogisticRegression(max_iter=1000), param_grid, cv=5)
grid_search.fit(X_train_tfidf, y_train)

# Access the mean test scores
mean_test_scores = grid_search.cv_results_['mean_test_score']
print(mean_test_scores)

# Find the best parameters and the corresponding mean test score
best_index = grid_search.best_index_
best_mean_test_score = mean_test_scores[best_index]
print("Best parameters:", grid_search.best_params_)
print("Best mean test score:", best_mean_test_score)


/Users/niveditasaha/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
15 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/niveditasaha/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/niveditasaha/anaconda3/lib/python3.11/site-packages/sklearn/base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/niveditasaha/anaconda3/lib/python3.11/site-packages/skle

[       nan 0.88948983        nan 0.9056393         nan 0.90591226]
Best parameters: {'C': 10, 'penalty': 'l2'}
Best mean test score: 0.9059122589459232


In [ ]:

# Parameters that were used in GridSearchCV
param_grid = {
    'C': [0.1, 1, 10],  # 'C' values
    'penalty': ['l2']   # Only 'l2' penalty as per the provided model output
}

# Placeholder for the mean test scores from GridSearchCV

mean_test_scores = np.array([0.89, 0.91, 0.9059]).reshape(1, -1)  

# Visualization of the heatmap
plt.figure(figsize=(8, 3))  # Size adjusted for a single row
sns.heatmap(mean_test_scores, annot=True, cmap='viridis', fmt='.4f',
            xticklabels=param_grid['C'], yticklabels=['l2'],
            cbar_kws={'label': 'Mean Test Score'})
plt.title('Grid Search Results')
plt.xlabel('C (Regularization Strength)')
plt.ylabel('Penalty Type')
plt.show()


In [ ]:

# Evaluate the best model on the test set
test_accuracy = best_model.score(X_test_tfidf, y_test)
print("Test Accuracy:", test_accuracy)


In [45]:
# Model Interpretability

# Get the coefficients (weights) of the features from the best logistic regression model
coefficients = best_model.coef_[0]

# Get the feature names from the TF-IDF vectorizer
feature_names = tfidf_vectorizer.get_feature_names_out()

# Create a DataFrame to store the coefficients and corresponding feature names
coefficients_df = pd.DataFrame({'Feature': feature_names, 'Coefficient': coefficients})

# Sort the DataFrame by coefficient magnitude to identify the most influential features
coefficients_df = coefficients_df.reindex(coefficients_df['Coefficient'].abs().sort_values(ascending=False).index)

# Display the top N most influential features
top_n = 10
print(f"Top {top_n} most influential features:")
print(coefficients_df.head(top_n))


Top 10 most influential features:
         Feature  Coefficient
3741    unwelcom    19.048129
1566      groggi    17.958683
2945      rotten    16.808778
1898        jade    16.802529
2062    listless    16.507131
3717    unimport    16.211394
1503      gloomi    15.901485
987     disillus    15.547955
986   dishearten    15.495591
2032     letharg    15.465748


In [46]:
# Plot the coefficients using a horizontal bar plot
top_n = 10

plt.figure(figsize=(10, 6))
sns.barplot(x='Coefficient', y='Feature', data=coefficients_df.head(top_n), palette='viridis')
plt.title('Top Features by Coefficient Magnitude')
plt.xlabel('Coefficient')
plt.ylabel('Feature')
plt.show()


# END